# **Launch Sites Locations Analysis with Folium**


## Objectives


Let's first import required Python packages for this lab:


In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
import requests
import io
import pandas as pd

# URL of the CSV file
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Use requests to fetch the data
resp = requests.get(URL)

# Convert the response content to a BytesIO object
spacex_csv_file = io.BytesIO(resp.content)

# Load the CSV into a pandas DataFrame
spacex_df = pd.read_csv(spacex_csv_file)

# Display the first few rows
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Iterate through each launch site in the dataframe
for index, site in launch_sites_df.iterrows():
    site_coordinate = [site['Lat'], site['Long']]  # Extract latitude and longitude
    site_name = site['Launch Site']  # Extract the site name
    
    # Add a circle around the launch site
    folium.Circle(
        site_coordinate,  # Coordinates for the circle
        radius=1000,  # Radius of 1 km
        color='#d35400',  # Circle color
        fill=True,
        fill_opacity=0.6
    ).add_child(folium.Popup(site_name)).add_to(site_map)  # Add popup with site name
    
    # Add a marker for the launch site
    folium.Marker(
        site_coordinate,  # Coordinates for the marker
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name,  # Label with site name
        )
    ).add_to(site_map)
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Define a function to assign colors based on class values
def assign_marker_color(launch_class):
    if launch_class == 1:
        return 'green'  # Successful launch
    else:
        return 'red'    # Failed launch

# Apply the function to create a new column 'marker_color'
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Check the new dataframe
spacex_df.head(20)

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [17]:
import math

In [18]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Import folium and MousePosition plugin
from folium.plugins import MousePosition

# Coordinates of the launch site (e.g., CCAFS SLC-40)
launch_site_coords = [28.56342, -80.57674]

# Create a folium map centered at the launch site
launch_map = folium.Map(location=launch_site_coords, zoom_start=10)

# Add MousePosition to the map to get coordinates when you click on a point
mouse_position = MousePosition()
launch_map.add_child(mouse_position)

# Display the map
launch_map

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of Earth in kilometers
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

# Coordinates of the closest coastline point
coastline_coords = [28.56367, -80.57163]

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_coords[0], launch_site_coords[1], coastline_coords[0], coastline_coords[1])

# Output the result
print(f"Distance from launch site to the closest coastline: {distance_coastline:.2f} km")


Distance from launch site to the closest coastline: 0.50 km


In [28]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.features import DivIcon

# Coordinates of the launch site and closest coastline
launch_site_coords = [28.56342, -80.57674]  # Example: CCAFS SLC-40
coastline_coords = [28.56367, -80.57163]    # Example coastline coordinates

# Step 1: Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_coords[0], launch_site_coords[1], coastline_coords[0], coastline_coords[1])

# Step 2: Create a folium map centered at the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Step 3: Add a folium.Marker on the closest coastline point
distance_marker = folium.Marker(
    coastline_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Step 4: Add the marker to the map
distance_marker.add_to(site_map)

# Step 5: Add a Circle marker for the launch site for better visualization
launch_site_marker = folium.CircleMarker(
    launch_site_coords,
    radius=10,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
)

# Add the launch site marker to the map
launch_site_marker.add_to(site_map)

# Display the map
site_map


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [29]:
import folium
from folium.features import DivIcon

# Coordinates for closest city, railway, and highway
city_coords = [28.3922, -80.6077]
railway_coords = [28.5721, -80.5853]
highway_coords = [28.5631, -80.5707]
launch_site_coords = [28.56342, -80.57674]  # Example: CCAFS SLC-40

# Step 1: Calculate distances
distance_city = calculate_distance(launch_site_coords[0], launch_site_coords[1], city_coords[0], city_coords[1])
distance_railway = calculate_distance(launch_site_coords[0], launch_site_coords[1], railway_coords[0], railway_coords[1])
distance_highway = calculate_distance(launch_site_coords[0], launch_site_coords[1], highway_coords[0], highway_coords[1])

# Step 2: Create a folium map centered at the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Step 3: Add markers with distances
# Marker for closest city
city_marker = folium.Marker(
    city_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM to City</b></div>'.format(distance_city),
    )
)

# Marker for closest railway
railway_marker = folium.Marker(
    railway_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM to Railway</b></div>'.format(distance_railway),
    )
)

# Marker for closest highway
highway_marker = folium.Marker(
    highway_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM to Highway</b></div>'.format(distance_highway),
    )
)

# Step 4: Add the markers to the map
city_marker.add_to(site_map)
railway_marker.add_to(site_map)
highway_marker.add_to(site_map)

# Add a Circle marker for the launch site for better visualization
launch_site_marker = folium.CircleMarker(
    launch_site_coords,
    radius=10,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
)

# Add the launch site marker to the map
launch_site_marker.add_to(site_map)

# Display the map
site_map


In [21]:
import folium
from folium.features import DivIcon

# Coordinates for closest city, railway, and highway
city_coords = [28.3922, -80.6077]
railway_coords = [28.5721, -80.5853]
highway_coords = [28.5631, -80.5707]
launch_site_coords = [28.56342, -80.57674]  # Example: CCAFS SLC-40

# Step 1: Calculate distances
distance_city = calculate_distance(launch_site_coords[0], launch_site_coords[1], city_coords[0], city_coords[1])
distance_railway = calculate_distance(launch_site_coords[0], launch_site_coords[1], railway_coords[0], railway_coords[1])
distance_highway = calculate_distance(launch_site_coords[0], launch_site_coords[1], highway_coords[0], highway_coords[1])

# Step 2: Create a folium map centered at the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Step 3: Add markers with distances
# Marker for closest city
city_marker = folium.Marker(
    city_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM to City</b></div>'.format(distance_city),
    )
)

# Marker for closest railway
railway_marker = folium.Marker(
    railway_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM to Railway</b></div>'.format(distance_railway),
    )
)

# Marker for closest highway
highway_marker = folium.Marker(
    highway_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM to Highway</b></div>'.format(distance_highway),
    )
)

# Step 4: Add the markers to the map
city_marker.add_to(site_map)
railway_marker.add_to(site_map)
highway_marker.add_to(site_map)

# Add a Circle marker for the launch site for better visualization
launch_site_marker = folium.CircleMarker(
    launch_site_coords,
    radius=10,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
)

# Add the launch site marker to the map
launch_site_marker.add_to(site_map)

# Step 5: Draw lines between the launch site and each proximity
# Line to closest city
folium.PolyLine(
    locations=[launch_site_coords, city_coords],  # Line from launch site to city
    color='blue'
).add_to(site_map)

# Line to closest railway
folium.PolyLine(
    locations=[launch_site_coords, railway_coords],  # Line from launch site to railway
    color='green'
).add_to(site_map)

# Line to closest highway
folium.PolyLine(
    locations=[launch_site_coords, highway_coords],  # Line from launch site to highway
    color='red'
).add_to(site_map)

# Display the map
site_map
